# Word2Vec

Word2vec is used for embedding words of a sentence. It basically tries to predict the next word given the context or surronding words. So in this file we are going to train the word2vec model and takes its hidden layer weights for use in embedding of words. 

## Building Word2Vec model

To build a word2vec model we need the sentences that is being used. The sentences are to be tokenized for use. Preprocessing of the text is allowed. Word2vec trains weights of the hidden layer which is later used as the vector representation of a given word.

Word2Vec is imported from gensim package. Word2vec uses two implementation 1. CBOW and 2. Skip-gram

Different methods of creating the initial corpus was tried out before building the model.

For Building a word2vec model the function word2vec is used where the sentences are passed as argument. Additional arguments are needed for specifying the number of cores that will be used for building the model 'workers' and the number of iterations.

In [1]:
from gensim.models import word2vec
#import logging
#logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

def trainW2Vmodel(modelData, modelName, iteration=50):
    print 'Training..'
    model = word2vec.Word2Vec(modelData , workers=4, iter = iteration)  #training of w2v model
    print 'Saving model..'
    model.save("w2v_models/"+modelName)          #to save the model
    print model.wv.most_similar("receive")            #to check to see the similar words 

--------------
## 1.
During the beginning the corpus was built from the xml file. The sentences were taken as it is without any mentioning of BR. Only some preprocessing was done like removing common stopwords like 'the','a'.

Here the file is read and each column data is stored.

In [2]:
fileName = "WhiteText"               #the file to be read

import xml.etree.ElementTree as ET
tree = ET.parse('data/'+fileName+'_re.xml')
root = tree.getroot()
print root.tag                       #Checking the tag of the root to see if file was read

corpus


The xml file is read and each sentence is stored into a list to be given as input for building model.

In [3]:
originalSen = []
for corpus in root:
    for document in corpus:
        sentence = document.attrib['text']
        originalSen.append(sentence.split())    #word2vec model takes words individually from a sentence. So it is required to split the sentence into tokens.
modelName  = 'xmlOriginalSen'

The input data is then given to the model to be trained and the model is saved for later use for vectorization of words.

In [4]:
trainW2Vmodel(originalSen, modelName)

Training..
Saving model..
[('receives', 0.6545082926750183), ('received', 0.6069025993347168), ('convergent', 0.6043736338615417), ('originated', 0.5531840920448303), ('Other', 0.5450671315193176), ('arise', 0.5387920141220093), ('neocortical', 0.5304872989654541), ('send', 0.5190256237983704), ('corticofugal', 0.5061150789260864), ('Brainstem', 0.48370733857154846)]


-----------------
### 2.
The next method was to replace the brain region mentions with BR's tags. No differentiation between any two BR. This was done when the thought process was thinking that for building the vector embedding there is no need for storing brain region mentions as they are different for each sentence. But as we are trying to build around the context of the BR, this seemed like a good idea.

For a start all brain region mention was denoted as BR. For that we read the xml file. Some preprocessing is done again before building of model.

Words like 'the','a' are common words called stopwords. These were removed. Other stopwords are useful in generating a context that talked about a conneciton, so stopword list was not used to remove all the stopwords.

Words like 'and','or' commonly occurred with a list of comma seperated BR's. So these words were removed for the purpose of creating a set of BR occuring together.

In [5]:
fileName = "WhiteText"               #the file to be read
modelName = "xmlReplaceBR"

import re
import xml.etree.ElementTree as ET
tree = ET.parse('data/'+fileName+'_re.xml')
root = tree.getroot()

sentences = []
for corpus in root:
    for document in corpus:
        sentence = document.attrib['text']
        
        for connection in document:
            if connection.tag == 'entity':
                entity = connection.attrib['text']
                sentence = sentence.replace(entity,"BR")
        
        sentence = re.sub(" (the|a) "," ",sentence)        #common words removed
        sentence = re.sub(" (and|or) "," ",sentence)       #to remove and, or instances 
        sentence = re.sub(",","", sentence)                # removing the commas 
        sentence = re.sub("\(((?!( BR (.*?))).*?)\)","",sentence)   #Words inside brackets not considered.
        sentences.append(sentence.split()) 
        
trainW2Vmodel(sentences, modelName)

Training..
Saving model..
[('receives', 0.6774589419364929), ('originated', 0.6198529005050659), ('arise', 0.598916232585907), ('send', 0.5885165333747864), ('sources', 0.565379798412323), ('receiving', 0.5652183294296265), ('demonstrate', 0.5508649349212646), ('convergent', 0.5469331741333008), ('originate', 0.5309138298034668), ('Other', 0.5291444063186646)]


-------
### 3.
Another word2vec corpus built was from the csv(1) file. 

The difference between the above the this method is that there are sentences with more than 2 brain region mentions. So sentence was expanded accordingly where each sentence had to be marked by BR1 and BR2.

Each sentence was being considerd different with BR1 and BR2 mentions. The sentence was read from the csv file and the corresponding BR regions were denoted by BR1 and BR2.

The csv file read here.

In [6]:
import pandas as pd
import re

fileName = "WhiteText(1).csv"
modelName = "csvReplaceBR12"

data = pd.read_csv('data/'+fileName,delimiter="|")

sentences = data["sentence"]
entity1 = data["entity2"]
entity2 = data["entity1"]

Now we replace the brain region mentions in the sentence with BR1 and BR2 corresponding to entity1 and entity2 respectively. Preprocessing was done as done with xml file before. The sentences ,after replacing brain regions with BR, was tokenized as word2vec input needs to be tokenized.

In [7]:
replaceSen = []

for i,sentence in enumerate(sentences):
    sentence = sentence.replace(entity1[i]," BR1 ")
    sentence = sentence.replace(entity2[i]," BR2 ")
    sentence = re.sub(" (the|a) "," ",sentence)        
    sentence = re.sub(" (and|or) "," ",sentence)       
    sentence = re.sub(",","", sentence)                
    sentence = re.sub("\(((?!( BR (.*?))).*?)\)","",sentence)   
    replaceSen.append(sentence.split())
    
trainW2Vmodel(replaceSen,modelName)

Training..
Saving model..
[('receives', 0.5293567776679993), ('received', 0.4618937075138092), ('arise', 0.44799894094467163), ('Amygdaloid', 0.39426302909851074), ('Origin', 0.39122605323791504), ('forward', 0.36313915252685547), ('gate', 0.35809195041656494), ('observe', 0.35645490884780884), ('respond', 0.35544753074645996), ('elasmobranchs', 0.35395199060440063)]


### 4.
The problem till now was that for tagging BR in the sentence every occurrence of a particular entity was replaced, but in a sentence there was a possibility that there may be multiple occurence of a single entity but only one of them would actually come in the connection declaring sentence.

So using the charOffset attribute in the xml file, BR1 and BR2 tagged appropriately and csv(2) file is used here. Words inside brackets were considered as there were many BRs inside them that showed connection.

In [8]:
import pandas as pd
import re

fileName = "WhiteText(2).csv"
modelName = "csv_2_ReplaceBR12"

data = pd.read_csv('data/'+fileName,delimiter="|")

sentences = data["sentence"]

replaceSen = []

for i,sentence in enumerate(sentences):
    sentence = re.sub(" (the|a) "," ",sentence)        
    sentence = re.sub(" (and|or) "," ",sentence)       
    sentence = re.sub(",","", sentence)                  
    replaceSen.append(sentence.split())
    
trainW2Vmodel(replaceSen,modelName)

Training..
Saving model..
[('receives', 0.45849576592445374), ('received', 0.4462020993232727), ('perhaps', 0.40566664934158325), ('arise', 0.4042736887931824), ('gate', 0.3975135087966919), ('(ZI).', 0.379808247089386), ('originating', 0.34888631105422974), ('contribute', 0.3299430012702942), ('provide', 0.32711732387542725), ('innervate', 0.3254753649234772)]


___________
### 5.
In a sentence there were many occurence of different brain regions which made the sentence lengthy without given much information. So we backtracked to where all the entities were tagged as BR but this time using charOffset attribute available. csv(3) file was made previous for this purpose.

In [9]:
import pandas as pd
import re

fileName = "WhiteText_re(3).csv"
modelName = "csv_3_ReplaceBR"

data = pd.read_csv('data/'+fileName,delimiter="|")

sentences = data["sentence"]

replaceSen = []

for i,sentence in enumerate(sentences):
    sentence = re.sub(" (the|a) "," ",sentence)        
    sentence = re.sub(" (and|or) "," ",sentence)       
    sentence = re.sub(",","", sentence)                  
    replaceSen.append(sentence.split())
    
trainW2Vmodel(replaceSen,modelName)

Training..
Saving model..
[('receives', 0.69260174036026), ('send', 0.610828161239624), ('supply', 0.5953450202941895), ('convergent', 0.5866342782974243), ('sources', 0.5709414482116699), ('arise', 0.5700213313102722), ('originated', 0.5657258033752441), ('somatosensory', 0.54510098695755), ('corticofugal', 0.5376680493354797), ('receiving', 0.5315340757369995)]


___________
### 6.
There were occurence of phrases that were common but were considered seperately for building a model. The process of combining two or more words together to form a single phrase is called n-gram model. 

So using the built-in funciton of gensim itself, phrases were included and model built.

In [10]:
from gensim.models import Phrases
import pandas as pd
import re

fileName = "WhiteText_re(3).csv"
modelName = "csv_3_ReplaceBR_bigram"

data = pd.read_csv('data/'+fileName,delimiter="|")

sentences = data["sentence"]

replaceSen = []

for i,sentence in enumerate(sentences):
    sentence = re.sub(" (the|a) "," ",sentence)        
    sentence = re.sub(" (and|or) "," ",sentence)
    sentence = re.sub("\([0-9]\)",".",sentence)
    sentence = re.sub(",","", sentence)                  
    replaceSen.append(sentence.split())
    

bigram_trans = Phrases(replaceSen)               #here the phrases are picked out a class built on top of sentence s
trainW2Vmodel(bigram_trans[replaceSen],modelName)  #the input is the sentences itself with phrases denoted.

/home/tsk/tensorflow/local/lib/python2.7/site-packages/gensim/models/phrases.py:494: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


Training..
Saving model..
[(u'receives', 0.7620521187782288), (u'convergent', 0.7129522562026978), (u'receives_its', 0.6716634631156921), (u'supply', 0.670750617980957), (u'type_2(HSD2)', 0.6601485013961792), (u'input_from', 0.6518070697784424), (u'preganglionic_parasympathetic', 0.6487269401550293), (u'BR(BPN)', 0.6421033143997192), (u'input.', 0.6412110328674316), (u'except_for', 0.6349889636039734)]


### 7. 
Bigram transformation used where BR1 and BR2 was tagged appropriately. Neccessary preprocessing was done.

In [11]:
import pandas as pd
import re

fileName = "WhiteText(2).csv"
modelName = "csv_2_ReplaceBR12_bigram"

data = pd.read_csv('data/'+fileName,delimiter="|")

sentences = data["sentence"]

replaceSen = []

for i,sentence in enumerate(sentences):
    sentence = re.sub(" (the|a) "," ",sentence)        
    sentence = re.sub("(nucleus)"," ",sentence)
    sentence = re.sub(" (and|or) "," ",sentence)       
    sentence = re.sub(",","", sentence)                  
    sentence = re.sub("\([0-9]\)",".",sentence)
    replaceSen.append(sentence.split())
    
bigram_trans = Phrases(replaceSen)               
trainW2Vmodel(bigram_trans[replaceSen],modelName)

Training..
Saving model..
[(u'receives', 0.5433603525161743), (u'received', 0.47046300768852234), (u'possess_zincergic', 0.4185108542442322), (u'modulate', 0.4146653413772583), (u'(CN)_receives', 0.41382256150245667), (u'Origin', 0.3878823220729828), (u'provide', 0.3850170373916626), (u'receive_substantial', 0.37164539098739624), (u'lamina_I', 0.3599889576435089), (u'(apart', 0.3586433529853821)]


### 8.
There are many sentences were BR are grouped together like comma seperated list of BR's. This made the sentence lengthy and hard to understand. So BR's were grouped together. So 2 types of models were used. One for one gram and the other for bi-gram phrases.

In [12]:
import pandas as pd
import re

fileName = "WhiteText(2).csv"
modelName = "csv_2_groupBR12"

data = pd.read_csv('data/'+fileName,delimiter="|")

sentences = data["sentence"]

replaceSen = []

for i,sentence in enumerate(sentences):
    sentence = re.sub(" (the|a) "," ",sentence)        
    sentence = re.sub("(nucleus)"," ",sentence)
    sentence = re.sub(" (and|or) "," ",sentence)       
    sentence = re.sub(","," ", sentence)  
    sentence = re.sub("\([0-9]\)",".",sentence)
    sentence = re.sub("(BR[, ])([ ]*(BR[, ]+))+"," BR ",sentence)
    sentence = re.sub(" (BR[ ]+)*BR1([ ]+BR[^12])*"," BR1 ",sentence)
    sentence = re.sub(" (BR[ ]+)*BR2([ ]+BR[^12])*"," BR2 ",sentence)
    replaceSen.append(sentence.split())
    
trainW2Vmodel(replaceSen,modelName)

Training..
Saving model..
[('received', 0.4512922763824463), ('receives', 0.4449634552001953), ('BR(Po)', 0.379641592502594), ('arise', 0.37749427556991577), ('gate', 0.3767459988594055), ('(SG-Li)', 0.3544248938560486), ('provide', 0.3539482355117798), ('originating', 0.3480599522590637), ('(ZI).', 0.33798307180404663), ('perhaps', 0.3334388732910156)]


##### Bigram

In [13]:
import pandas as pd
import re

fileName = "WhiteText(2).csv"
modelName = "csv_2_groupBR12_bigram"

data = pd.read_csv('data/'+fileName,delimiter="|")

sentences = data["sentence"]

replaceSen = []

for i,sentence in enumerate(sentences):
    sentence = re.sub(" (the|a) "," ",sentence)        
    sentence = re.sub("(nucleus)"," ",sentence)
    sentence = re.sub(" (and|or) "," ",sentence)       
    sentence = re.sub(","," ", sentence)  
    sentence = re.sub("\([0-9]\)",".",sentence)
    sentence = re.sub("(BR[, ])([ ]*(BR[, ]+))+"," BR ",sentence)
    sentence = re.sub(" (BR[ ]+)*BR1([ ]+BR[^12])*"," BR1 ",sentence)
    sentence = re.sub(" (BR[ ]+)*BR2([ ]+BR[^12])*"," BR2 ",sentence)
    replaceSen.append(sentence.split())
    
bigram_trans = Phrases(replaceSen)               
trainW2Vmodel(bigram_trans[replaceSen],modelName)

Training..
Saving model..
[(u'receives', 0.5809202194213867), (u'modulate', 0.4433830976486206), (u'received', 0.423903226852417), (u'receive_substantial', 0.42366743087768555), (u'(SG-Li)', 0.4077766537666321), (u'provide', 0.3929670453071594), (u'send', 0.39192214608192444), (u'derived', 0.3894904851913452), (u'extend', 0.37163546681404114), (u'possess_zincergic', 0.37094998359680176)]


### 9.
Till now the sentences were tokenized using split function where the word was split if there was any space inbetween. This was not a good approach as if there were no spaces inbetween a bracket and a word it was considered as one. So the tokenizer package of nltk was used.

In [14]:
import pandas as pd
import re
from nltk.tokenize import word_tokenize

fileName = "WhiteText(2).csv"
modelName = "csv_2_groupBR12_token"

data = pd.read_csv('data/'+fileName,delimiter="|")

sentences = data["sentence"]

replaceSen = []

for i,sentence in enumerate(sentences):
    sentence = re.sub("\s(the|The|a)\s"," ",sentence)
    sentence = re.sub("^(the|The)","",sentence)
    sentence = re.sub("(nucleus)"," ",sentence)
    sentence = re.sub(r","," ", sentence)
    sentence = re.sub("\([0-9]\)",".",sentence)
    sentence = re.sub("(BR[^12])([ ]*(BR[^12]))+"," BR ",sentence)
    sentence = re.sub(" (BR[ ]+)*BR1([ ]+BR[^12])*"," BR1 ",sentence)
    sentence = re.sub(" (BR[ ]+)*BR2([ ]+BR[^12])*"," BR2 ",sentence)
    replaceSen.append(word_tokenize(sentence))
    
trainW2Vmodel(replaceSen,modelName)

Training..
Saving model..
[('received', 0.5053277015686035), ('receives', 0.5016341805458069), ('gate', 0.4351196885108948), ('arise', 0.4334292709827423), ('arises', 0.39790964126586914), ('provide', 0.37127119302749634), ('originate', 0.3651004433631897), ('sources', 0.3650605082511902), ('implying', 0.3519928455352783), ('derive', 0.34967297315597534)]


Bigram phrase

In [15]:
import pandas as pd
import re
from nltk.tokenize import word_tokenize

fileName = "WhiteText(2).csv"
modelName = "csv_2_groupBR12_bigramToken"

data = pd.read_csv('data/'+fileName,delimiter="|")

sentences = data["sentence"]

replaceSen = []

for i,sentence in enumerate(sentences):
    sentence = re.sub(" (the|a) "," ",sentence)        
    sentence = re.sub("(nucleus)"," ",sentence)
    sentence = re.sub(" (and|or) "," ",sentence)       
    sentence = re.sub(","," ", sentence)  
    sentence = re.sub("\([0-9]\)",".",sentence)
    sentence = re.sub("(BR[, ])([ ]*(BR[, ]+))+"," BR ",sentence)
    sentence = re.sub(" (BR[ ]+)*BR1([ ]+BR[^12])*"," BR1 ",sentence)
    sentence = re.sub(" (BR[ ]+)*BR2([ ]+BR[^12])*"," BR2 ",sentence)
    replaceSen.append(word_tokenize(sentence))
    
bigram_trans = Phrases(replaceSen)               
trainW2Vmodel(bigram_trans[replaceSen],modelName)

Training..
Saving model..
[(u'receives', 0.6198257803916931), (u'received', 0.5028239488601685), (u'modulate', 0.45436805486679077), (u'send', 0.43561047315597534), (u'receive_monosynaptic', 0.41312751173973083), (u'receive_substantial', 0.4119001626968384), (u'Origin', 0.39184439182281494), (u'originated', 0.389091432094574), (u'excitatory_glutamatergic', 0.3883330225944519), (u'receive_convergent', 0.3830053508281708)]


### 10.
After the grouping of BR's together, there were found to be many duplicates with the same structure. So during initial processing, duplicates were removed and then used further. csv(4) file is used for this.

Here BR's, BR1 and BR2 are tagged. Bi-gram phrases were seen to give better score so only bigram model trained.

In [18]:
import pandas as pd
import re
from nltk.tokenize import word_tokenize

fileName = "WhiteText_re(Fin).csv"
modelName = "csv_Fin_bigramToken"

data = pd.read_csv('data/'+fileName,delimiter="|")

sentences = data["sentence"]

replaceSen = []

for i,sentence in enumerate(sentences):
    sentence = re.sub("^(the|The)","",sentence)
    sentence = re.sub("(nucleus)"," ",sentence)
    sentence = re.sub(r","," ", sentence)
    sentence = re.sub("\([0-9]\)",".",sentence)
    replaceSen.append(word_tokenize(sentence.lower()))
    
bigram_trans = Phrases(replaceSen)               
trainW2Vmodel(bigram_trans[replaceSen],modelName)

Training..
Saving model..
[(u'receives', 0.594934344291687), (u'received', 0.5410398840904236), (u'send', 0.43824541568756104), (u'provide', 0.4217180609703064), (u'possess', 0.40293335914611816), (u'may_be', 0.3971779942512512), (u'modest', 0.38671210408210754), (u'periodic_clustered', 0.38661330938339233), (u'primary_sources', 0.38417255878448486), (u'stemming', 0.36386537551879883)]
